<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-:-addresses-with-street-nb" data-toc-modified-id="Data-:-addresses-with-street-nb-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data : addresses with street nb</a></span></li><li><span><a href="#Data-with-time-lapse" data-toc-modified-id="Data-with-time-lapse-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data with time lapse</a></span></li><li><span><a href="#Segments-start-end-with-time-lapse" data-toc-modified-id="Segments-start-end-with-time-lapse-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Segments start end with time lapse</a></span></li></ul></div>

In [61]:
import pandas as pd
import json
import re
import os
import folium
from folium.plugins import TimestampedGeoJson
import editdistance
import seaborn as sns
import numpy as np
import pandas as pd
from collections import Counter
from unidecode import unidecode
from datetime import datetime
from tqdm import tqdm_notebook as tqdm

from tqdm._tqdm_notebook import tqdm_notebook

tqdm_notebook.pandas(tqdm(desc='Progress'))

### Data : addresses with street nb

In [3]:
matched_adresses = pd.read_pickle('df_addressing_with_numbers.pickle')

In [5]:
len(matched_adresses)

198181

In [4]:
matched_adresses.head()

,document,annee,page,row,Nom,Métier,Rue,Numéro,X,Y
0,bpt6k6282019m,1855,144,0,Aaron,bronzes,Passage Choiseul,72,2.335772,48.868399
6,bpt6k6282019m,1855,144,51,Abrahams Junior (J.-D.),pierres fines,Rue de la Bourse,1,2.340098,48.869317
8,bpt6k6282019m,1855,144,59,Abt (H.G.) fils et Cie,fabricant,Rue de Richelieu,79,2.338668,48.869342
9,bpt6k6282019m,1855,144,60,Abt (Luc),fabricant,Rue de Richelieu,64,2.338542,48.868710
11,bpt6k6282019m,1855,144,68,Acard (Eugène),huissier,Rue de Richelieu,92,2.339362,48.870343


### Data with time lapse

In [853]:
cafes = pd.read_csv('subsets/cafes.csv')

In [854]:
cafes.head()

,Nom,Métier,Rue,Numéro,X,Y,de,à
0,(170). mely (G.),café limonadier,Rue Réaumur,111,2.343325,48.868127,1921,1921
1,(3€). agueau (P.),café restaurant,Rue de Choiseul,29,2.335875,48.870977,1921,1921
2,Adam-Bolondo (yve),café brasserie,Rue de Hanovre,8,2.335116,48.870227,1881,1881
3,Adams (Mime),café brasserie,Rue Saint Augustin,41,2.333839,48.868811,1885,1885
4,Adams (Mme),café brasserie,Rue Saint Augustin,41,2.333839,48.868811,1882,1884


In [855]:
cafes['date']  = pd.to_datetime(cafes.de, format='%Y')
cafes_dict = []

for index, row in limo.iterrows():
    #store coordinates
    X = row['X']
    Y = row['Y']
    time = row['date'].strftime('%Y')


    cafe = {
        'type': 'Feature',
            'geometry': {
                'type': 'Point',
                'coordinates':[X,Y] 
            },
            'properties': {
                'popup': row['Nom'], 
                'time': time, 
                'icon': 'circle', 
                'iconstyle': {
                    'fillColor': 'green',
                    'fillOpacity': 0.6,
                    'stroke': 'false',
                    'radius': 5
                },
                'style': {'weight': 0},
                'id': 'man'
            }
        }
    cafes_dict.append(cafe)
    
    

In [857]:
m = folium.Map(location=[48.866305, 2.338029], tiles='cartodbpositron', zoom_start=15)

plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': cafes_dict},
    period='P1M',
    add_last_point=True,
    auto_play=True,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY',
    time_slider_drag_update=True,
    duration='P1Y'
).add_to(m)

In [858]:
m

### Segments start end with time lapse

In [92]:
from glob import glob

In [93]:
geo_data = glob('../geotemporal data/*.pickle')

In [94]:
geo_data

['../geotemporal data/geotemp_data_WGS84.pickle',
 '../geotemporal data/simple_geotemp_data_WGS84.pickle']

In [764]:
geotemp = pd.read_pickle('../geotemporal data/simple_geotemp_data_WGS84.pickle')

In [765]:
len(geotemp)

191

In [766]:
geotemp.reset_index(inplace =True)
geotemp['NOM_ENTIER']=geotemp['NOM_ENTIER'].str.replace('-', ' ')

In [767]:
geotemp = geotemp[['NOM_ENTIER', 'WKT', 'date_nom', 'date_fin']]

In [768]:
geotemp.head()

,NOM_ENTIER,WKT,date_nom,date_fin
0,Impasse Saint Claude,"LINESTRING (2.34373078736353 48.8664368913173,...",1802,2020
1,Impasse Saint Pierre,"LINESTRING (2.34366952697704 48.8680412015817,...",1640,2020
2,Impasse de la Brasserie,"LINESTRING (2.33504011232187 48.863980272158,2...",1867,2020
3,Petite Rue Saint Pierre,"LINESTRING (2.34378044875237 48.8677664660456,...",1666,2020
4,Rue Baillif,"LINESTRING (2.33943369069248 48.8646464480446,...",1803,1912


In [769]:
geotemp['Linestring'] = geotemp['WKT'].str.split('(').apply(lambda x : x[-1])

In [770]:
geotemp['Linestring'] = geotemp['Linestring'].str.replace('(', '').str.replace(')','')

In [771]:
geotemp['Linestring']  = geotemp['Linestring'].str.split(',')

In [772]:
coords = []
for index, row in geotemp.iterrows():
    coords_list_str = row['Linestring']
    coords_list_float = []
    for coor in coords_list_str:
        x,y = coor.split()
        x_fl = float(x)
        y_fl = float(y)
        coords_list_float.append([x_fl,y_fl])
    
   # print(coords_list_float, 'brigitte', '\n')
    #row['coords'].append(coords_list_float)
    coords.append(coords_list_float)
    
m = np.asarray(coords)
geotemp['coords'] = m

In [773]:
geotemp.drop(columns=['WKT', 'Linestring'], inplace = True)

In [777]:
geotemp.head()

,NOM_ENTIER,date_nom,date_fin,coords,date_start_map,duration
0,Impasse Saint Claude,1802,2020,"[[2.34373078736353, 48.8664368913173], [2.3437...",1802,250
1,Impasse Saint Pierre,1640,2020,"[[2.34366952697704, 48.8680412015817], [2.3435...",1700,350
2,Impasse de la Brasserie,1867,2020,"[[2.33504011232187, 48.863980272158], [2.33508...",1867,200
3,Petite Rue Saint Pierre,1666,2020,"[[2.34378044875237, 48.8677664660456], [2.3436...",1700,350
4,Rue Baillif,1803,1912,"[[2.33943369069248, 48.8646464480446], [2.3395...",1803,150


For some reason, the transformation in date format is NaT for years earlier than 1676. I then decided to set years to 1700 at the earliest (-> the map starts at 1700 even for older streets).

In [775]:
geotemp['date_start_map'] = geotemp['date_nom']

mask = geotemp.date_nom < 1700
column_name = 'date_start_map'
geotemp.loc[mask, column_name] = 1700

Method : duplicate each street every 50 years and add 50 years to the start date each time

In [776]:
geotemp['duration'] = (((geotemp['date_fin'] - geotemp['date_start_map']) // 50)+1) * 50

In [780]:
geotemp50 = []
for idx, row in geotemp.iterrows():
    nom, date_nom, date_fin, coords, date_start_map, duration = row.values
    for i in range(duration//50):
        geotemp50.append((nom, date_nom, date_fin, coords, duration, date_start_map+i*50))
        
df_50 = pd.DataFrame(geotemp50, columns=['NOM_ENTIER', 'date_nom', 'date_fin', 'coords', 'duration', 'date_start_map'])

In [782]:
df_50['date_start_map'] = pd.to_datetime(df_50.date_start_map, format='%Y', errors='coerce')

In [831]:
df_50.head()

,NOM_ENTIER,date_nom,date_fin,coords,duration,date_start_map
0,Impasse Saint Claude,1802,2020,"[[2.34373078736353, 48.8664368913173], [2.3437...",250,1802-01-01
1,Impasse Saint Claude,1802,2020,"[[2.34373078736353, 48.8664368913173], [2.3437...",250,1852-01-01
2,Impasse Saint Claude,1802,2020,"[[2.34373078736353, 48.8664368913173], [2.3437...",250,1902-01-01
3,Impasse Saint Claude,1802,2020,"[[2.34373078736353, 48.8664368913173], [2.3437...",250,1952-01-01
4,Impasse Saint Claude,1802,2020,"[[2.34373078736353, 48.8664368913173], [2.3437...",250,2002-01-01


In [833]:
geo_dict = []

for index, row in df_50.iterrows(): 
    coor = row['coords']
    times = [row['date_start_map'].strftime('%Y')] * len(coor)
    street = {
        'type': 'Feature',
            'geometry': {
                'type': 'LineString',
                'coordinates': coor
            },
            'properties': {
                'times': times, # CAREFUL : "time" -> one value but "times" -> a list of values
                'popup': (row['NOM_ENTIER'] + " bâtie en " + str(row['date_nom']))

            } 
            }
        
    geo_dict.append(street)

In [834]:
len(geo_dict)

924

In [835]:
m = folium.Map(location=[48.866305, 2.338029], tiles='cartodbpositron', zoom_start=15)

plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': geo_dict},
    period='P1M',
    transition_time =10,
    add_last_point=True,
    auto_play=True,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options='YYYY',
    time_slider_drag_update=True,
    duration='P50Y' # each street is displayed during 10 years
).add_to(m)

In [836]:
m

In [558]:
help(folium.Icon)

Help on class Icon in module folium.map:

class Icon(branca.element.MacroElement)
 |  Creates an Icon object that will be rendered
 |  using Leaflet.awesome-markers.
 |  
 |  Parameters
 |  ----------
 |  color : str, default 'blue'
 |      The color of the marker. You can use:
 |  
 |          ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
 |           'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
 |           'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
 |           'gray', 'black', 'lightgray']
 |  
 |  icon_color : str, default 'white'
 |      The color of the drawing on the marker. You can use colors above,
 |      or an html color code.
 |  icon : str, default 'info-sign'
 |      The name of the marker sign.
 |      See Font-Awesome website to choose yours.
 |      Warning : depending on the icon you choose you may need to adapt
 |      the `prefix` as well.
 |  angle : int, default 0
 |      The icon will be rotated by this amount of degrees.
 